# **경사하강법을 이용한 Shallow Neural Network 학습**

In [2]:
import tensorflow as tf
import numpy as np

# 하이퍼 파라미터

In [19]:
epochs = 1000

# 네트워크 구조 정의
* Shallow Neural Network

input layer: 2 <br/>
hidden layer: 128 (Sigmoid activation) <br/>
output layer: 10 (Softmax activation) <br/>

In [20]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()

    self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x, training=None, mask=None):
    x = self.d1(x)  # hidden layer
    return self.d2(x) # output layer



# 학습 루프 정의

In [21]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  
  gradients = tape.gradient(loss, model.trainable_variables) # df(x)/dx

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_metric(labels, predictions)

# Dataset 생성

In [22]:
np.random.seed(0)

pts = []
labels = []
center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
  for _ in range(100):
    pts.append(center_pt + np.random.randn(*center_pt.shape))
    labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

# 모델 생성


In [23]:
model = MyModel()

# 손실 함수 및 최적화 알고리즘 설정
* Cross Entropy , Adam Optimizer

In [24]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 평가지표 설정
* Accuracy

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 학습 루프

In [27]:
for epoch in range(epochs):
  for x, label in train_ds:
    train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100))

Epoch 1, Loss: 0.4144861102104187, Accuracy: 87.08525848388672
Epoch 2, Loss: 0.41361236572265625, Accuracy: 87.09870910644531
Epoch 3, Loss: 0.4127576947212219, Accuracy: 87.11026000976562
Epoch 4, Loss: 0.41191476583480835, Accuracy: 87.12052917480469
Epoch 5, Loss: 0.41107600927352905, Accuracy: 87.13468933105469
Epoch 6, Loss: 0.41025376319885254, Accuracy: 87.14527130126953
Epoch 7, Loss: 0.4094386398792267, Accuracy: 87.15743255615234
Epoch 8, Loss: 0.40862464904785156, Accuracy: 87.16889953613281
Epoch 9, Loss: 0.40786945819854736, Accuracy: 87.1780014038086
Epoch 10, Loss: 0.40709444880485535, Accuracy: 87.18756103515625
Epoch 11, Loss: 0.4063121974468231, Accuracy: 87.19921112060547
Epoch 12, Loss: 0.40553009510040283, Accuracy: 87.20909881591797
Epoch 13, Loss: 0.40481239557266235, Accuracy: 87.22160339355469
Epoch 14, Loss: 0.40404409170150757, Accuracy: 87.22962951660156
Epoch 15, Loss: 0.4032876789569855, Accuracy: 87.24026489257812
Epoch 16, Loss: 0.40252918004989624, Acc

# 데이터셋 및 학습 파라미터 저장

In [29]:
np.savez_compressed('dataset.npz', inputs=pts, labels=labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('parameters.npz', 
                    W_h = W_h,
                    b_h = b_h,
                    W_o = W_o,
                    b_o = b_o)